# Cкользящее среднее

Интуитивно понятный способ превратить непонятную «пилу» в график, на котором можно что-то увидеть — это **скользящее среднее** или *Moving Average (MA)*. Проще всего взять среднее арифметическое последних двух наблюдений. Мы получим новый временной ряд, каждый член которого — среднее арифметическое двух соседних значений исходного ряда

$ MA_t=\frac{X_{t-1}+X_t}{2} $

Чуть более продвинутый способ: усреднить сразу несколько наблюдений. Это так называемое **простое скользящее среднее** *(SMA)*

$ SMA_t=\frac{X_{t-k}+…+X_t}{k} $

Продвинутым вариантом является **взвешенное скользящее среднее** *(WMA)* последовательных наблюдений ряда

$ WMA_t=\sum_{i=0}^{k-1}w_{t-i}X_{t-i} $

Сумма весов при этом равна единице

$ \sum_{i=0}^{k-1}w_{t-i}=1 $

Как правило, при таком усреднении чем новее наблюдение, чем больше его вес.

In [1]:
import numpy as np

In [13]:
def weighted_moving_average(x, w):
    return np.convolve(x, w, 'valid')

In [20]:
def moving_average(x, w):
    weights = np.full(w, 1/w)
    return weighted_moving_average(x, weights)

Вычислите значения простого скользящего среднего для ряда $ x_0=1, x_1=2, x_2=3, x_3=4, x_4=5, x_5=6 $

In [21]:
x = np.array([1, 2, 3, 4, 5, 6])
moving_average(x, 2)

array([1.5, 2.5, 3.5, 4.5, 5.5])

Вычислите значения скользящего среднего по правилу $ MA_t=0.7x_t+0.3x_{t-1} $ для ряда $ x_0=1, x_1=2, x_2=3, x_3=4, x_4=5, x_5=6 $

In [22]:
x = np.array([1, 2, 3, 4, 5, 6])
w = np.array([0.7, 0.3])

weighted_moving_average(x, w)

array([1.7, 2.7, 3.7, 4.7, 5.7])

## Задание 5.5

Для ряда $ x=(1, 0, 1, -1, 2, 0, 1) $. Вычислите ряд скользящих средних с шириной окна $3$ и весами $0.5, 0.3, 0.2$.

In [23]:
x = np.array([1, 0, 1, -1, 2, 0, 1])
w = np.array([0.5, 0.3, 0.2])

weighted_moving_average(x, w)

array([ 0.7, -0.2,  0.9,  0.4,  0.9])

## 6. Практика. Скользящее среднее

А теперь попрактикуемся с помощью данных об активности солнечных вспышек. Загрузите датасет `solarpower_cumuldaybyday2.csv`. В нём содержится информация о производстве энергии солнечными батареями. Агрегируйте данные по неделям.

In [28]:
import pandas as pd

In [55]:
df = pd.read_csv('data/solarpower_cumuldaybyday2.csv', usecols=['date', 'cum_power'])
df.head(5)

,date,cum_power
0,26/10/2011,0.1
1,27/10/2011,10.2
2,28/10/2011,20.2
3,29/10/2011,29.6
4,30/10/2011,34.2


In [56]:
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')

Постройте ряд еженедельного производства энергии, вычислив приращения полученного недельного ряда. В поле для ответа введите значение произведенной энергии за неделю с 6 по 12 октября 2014, округлив до целых.

In [66]:
df['diff_power'] = df['cum_power'].diff()

In [119]:
weekly = df.groupby([pd.Grouper(key='date', freq='W')]).apply(sum)

In [130]:
weekly['2014-10']

,cum_power,diff_power
date,,
2014-10-05,81567.0,61.0
2014-10-12,81898.0,44.0
2014-10-19,82223.0,49.0
2014-10-26,82532.0,33.0


In [134]:
x = weekly['2014-10'].iloc[1, 1]
print(f'значение произведенной энергии за неделю с 6 по 12 октября 2014: {x:.0f}')

значение произведенной энергии за неделю с 6 по 12 октября 2014: 44


Постройте ряд скользящего среднего для размера окна 4. Сколько элементов в ряде скользящего среднего? 

In [152]:
source_items = weekly['diff_power'].shape[0]
average_items = weekly['diff_power'].rolling(window=4).mean().shape[0]

print(f'Столько же, сколько в исходном: {source_items == average_items}')
print(f'Меньше, чем в исходном: {source_items > average_items}')

Столько же, сколько в исходном: True
Меньше, чем в исходном: False


Сколько числовых элементов в ряде скользящего среднего?

In [153]:
weekly['diff_power'].rolling(window=4).mean().dropna().shape[0]

363

Найдите значение ряда скользящего среднего с шириной окна $4$ для $t=120$.

In [154]:
weekly['diff_power'].rolling(window=4).mean()[119]

21.5